# 🚦 Traffic Manager AI (V12 - Fixed)
### Fix reconnect + hỗ trợ đèn giao thông

In [ ]:
# Cell 1: Dependencies + Model
!pip install ultralytics python-socketio[client] websocket-client opencv-python-headless --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt
# Nếu có model đèn giao thông, upload lên Kaggle và đổi tên file bên dưới

import torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Loading on {device}...")

# Vehicle model
vehicle_model = YOLO('yolo11m.pt').to(device)
print("✅ Vehicle model loaded!")

# Traffic light model (uncomment nếu có)
# tl_model = YOLO('mhiot-dentinhieu-best-new.pt').to(device)
# print("✅ Traffic light model loaded!")
tl_model = None

In [ ]:
# Cell 2: Main Loop (Fixed reconnect)
import cv2, numpy as np, socketio, base64, time, threading, queue, logging, warnings
from datetime import datetime

# ========== CẬP NHẬT URL ==========
SERVER_URL = 'https://liberal-surrounding-lease-estimates.trycloudflare.com'
# ==================================

logging.getLogger('socketio').setLevel(logging.WARNING)
logging.getLogger('engineio').setLevel(logging.WARNING)
warnings.filterwarnings('ignore')

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
CONFIDENCE = 0.5

# Single socket instance - NO duplicate connections
sio = socketio.Client(
    reconnection=True,
    reconnection_attempts=0,
    reconnection_delay=1,
    reconnection_delay_max=3,
    ssl_verify=False,
    logger=False,
    engineio_logger=False
)

queues = {}
running = True
stats = {'f': 0, 'd': 0, 'c': 0, 'dc': 0}
connecting = False  # Prevent duplicate connect calls

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def process(cid):
    log(f"📹 {cid[-4:]} started")
    while running:
        try:
            data = queues[cid].get(timeout=0.5)
            if data is None: continue
            
            f, c, i, t = data
            stats['f'] += 1
            h, w = f.shape[:2]
            dets = []
            
            # Vehicle detection
            for r in vehicle_model(f, verbose=False, conf=CONFIDENCE):
                for b in r.boxes:
                    cls = vehicle_model.names[int(b.cls[0])]
                    if cls not in VEHICLE_CLASSES: continue
                    x1,y1,x2,y2 = map(int, b.xyxy[0])
                    dets.append({'type':'vehicle','class':cls,'confidence':float(b.conf[0]),
                        'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}})
            
            # Traffic light detection (nếu có model)
            if tl_model is not None:
                for r in tl_model(f, verbose=False, conf=CONFIDENCE):
                    for b in r.boxes:
                        cls = tl_model.names[int(b.cls[0])]
                        x1,y1,x2,y2 = map(int, b.xyxy[0])
                        dets.append({'type':'traffic_light','class':cls,'confidence':float(b.conf[0]),
                            'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}})
            
            if dets:
                stats['d'] += len(dets)
                if sio.connected:
                    sio.emit('car_detected', {'camera_id':c,'image_id':i,'detections':dets,'created_at':t})
                    log(f"✅ {c[-4:]}: {len(dets)} | F:{stats['f']} D:{stats['d']}")
            
            time.sleep(0.05)
        except queue.Empty:
            continue
        except Exception as e:
            log(f"❌ Process: {e}")

@sio.event
def connect():
    global connecting
    connecting = False
    stats['c'] += 1
    log(f"✅ CONNECTED #{stats['c']}")
    sio.emit('join_all_camera')

@sio.event
def disconnect():
    stats['dc'] += 1
    log(f"⚠️ DISCONNECTED #{stats['dc']} - auto reconnecting...")

@sio.on('image')
def on_image(d):
    try:
        img = d.get('image') or d.get('buffer')
        if isinstance(img, dict): img = bytes(img.get('data', []))
        if isinstance(img, str): img = base64.b64decode(img)
        f = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
        if f is None: return
        
        c = d['cameraId']
        if c not in queues:
            queues[c] = queue.Queue(maxsize=3)
            threading.Thread(target=process, args=(c,), daemon=True).start()
        
        q = queues[c]
        # Clear old frames on reconnect
        while not q.empty():
            try: q.get_nowait()
            except: break
        q.put((f, c, d['imageId'], d.get('created_at', 0)))
    except Exception as e:
        log(f"❌ Image: {e}")

def initial_connect():
    global connecting
    while running and not sio.connected:
        if not connecting:
            connecting = True
            log(f"🔄 Connecting to {SERVER_URL}...")
            try:
                sio.connect(SERVER_URL, transports=['websocket'])
            except Exception as e:
                log(f"❌ {e}")
                connecting = False
                time.sleep(2)
        else:
            time.sleep(0.5)

# Start
log(f"🎯 {SERVER_URL}")
threading.Thread(target=initial_connect, daemon=True).start()

try:
    while running:
        time.sleep(60)
        log(f"📊 F:{stats['f']} D:{stats['d']} C:{stats['c']} DC:{stats['dc']}")
except:
    running = False
    sio.disconnect()